# Tools and Routing

In [23]:
# !pip install pydantic==1.10.8

# !pip install wikipedia

In [1]:
from langchain.agents import tool

In [2]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [3]:
search.name

'search'

In [4]:
search.description

'search(query: str) -> str - Search for weather online'

In [5]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [6]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [7]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [8]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [9]:
search.run("sf")

'42f'

In [10]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [11]:
get_current_temperature.name

'get_current_temperature'

In [12]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [13]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [14]:
from langchain.tools.render import format_tool_to_openai_function

In [15]:
format_tool_to_openai_function(get_current_temperature)

c:\Users\kaila\DLAI\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [17]:
get_current_temperature({"latitude": 19, "longitude": 72})

'The current temperature is 30.2°C'

In [18]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [19]:
search_wikipedia.name

'search_wikipedia'

In [20]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [21]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [24]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which e

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [31]:
from langchain_community.chat_models import ChatOllama

functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOllama(model="phi3:instruct", temperature=0).bind(functions=functions)

In [32]:
model.invoke("what is the weather in sf right now")

AIMessage(content=' As an AI, I\'m unable to provide real-time data. However, you can easily check the current weather in San Francisco by visiting a reliable weather forecasting website like Weather.com or using a weather app on your smartphone. Simply search for "San Francisco weather" to get up-to-date information.', response_metadata={'model': 'phi3:instruct', 'created_at': '2024-06-02T13:51:56.267523Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 14923581500, 'load_duration': 6938136600, 'prompt_eval_count': 12, 'prompt_eval_duration': 562273000, 'eval_count': 72, 'eval_duration': 7413778000}, id='run-c368f3a2-e678-49bd-90a4-e0aeb6cbd8d8-0')

In [34]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query":"Langchain"}'}})

In [35]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [36]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":37.7749,"longitude":-122.4194}'}})

In [37]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [38]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [39]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [40]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [41]:
result.tool

'get_current_temperature'

In [42]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [43]:
get_current_temperature(result.tool_input)

'The current temperature is 20.9°C'

In [44]:
result = chain.invoke({"input": "hi!"})

In [45]:
type(result)

langchain.schema.agent.AgentFinish

In [46]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

In [47]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [48]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [49]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [50]:
result

'The current temperature is 20.9°C'

In [51]:
result = chain.invoke({"input": "What is langchain?"})

In [52]:
result

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring an instruction that can be interpreted and understood by a generative AI model. A prompt is natural

In [53]:
chain.invoke({"input": "hi!"})

'Hello! How can I assist you today?'